In [1]:
#Estatísticos
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

#Manipulação de Dados
import pandas as pd
import numpy as np
from numpy.linalg import LinAlgError

#Utilitários
import pickle
import itertools
from datetime import date, timedelta
from joblib import Parallel, delayed
import warnings

#Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

#Ajustes dos Gráficos
rcParams['figure.figsize'] = 18, 12

from Fase3_Aux_Func import (agrupamento_periodico,
                            ajuste_datas_historica,
                            conversao_array,
                            plot_serie,
                            plot_decomposicao,
                            suavizacao_simples,
                            suavizacao_exponencial,
                            holt_winters,
                            arma,
                            arima_sazonal,
                            analise_residuos,
                            check_stationarity,
                            grid_search_best_model,
                            grid_search_best_model_no_parallel)


# Suprimir todos os warnings
warnings.filterwarnings('ignore')

In [3]:
global local_origem 
global data_corte_treinamento

local_origem = "/home/cdsw"
data_corte_treinamento = '2024-06-30'

### 1. Carregramento dos Dados

In [4]:
'''
LEMBRA DE MUDAR O NOME DO ARQUIVO PARA df_final_tratado DEPOIS DE AUTALIZAR O BD_BILHETAGEM E RODAR O DF TRATADO
'''
df_bilhetagem = pd.read_csv(local_origem + "/df_final_agrupado_ano_mes.csv", sep=",", decimal="," )
df_bilhetagem['Volume_Passageiros_BU_VT_DIN'] = df_bilhetagem['Volume_Passageiros_BU_VT_DIN'].astype(int)
df_bilhetagem.head()

,Ano,Mes,Zona,Sub_Prefeitura,Volume_Passageiros_BU_VT_DIN
0,2014,1,CENTRO,SE,19599200
1,2014,1,LESTE,ARICANDUVA-FORMOSA-CARRAO,1234500
2,2014,1,LESTE,ERMELINO MATARAZZO,37373
3,2014,1,LESTE,GUAIANASES,2652418
4,2014,1,LESTE,ITAIM PAULISTA,623072


In [5]:
'''
Irá realizar os ajustes de datas necessários, periorizando por dia, além de quebrar em 2 arquivos
'''
df_subpref, df_zonas = agrupamento_periodico(df_bilhetagem)

In [6]:
#Criação das Listas
Sub_Prefeituras = np.unique(df_bilhetagem['Sub_Prefeitura'].astype(str).to_list())
Zonas = np.unique(df_bilhetagem['Zona'].astype(str).to_list())

### 2. Ajuste dos Modelos (Grid Search)

In [9]:
# Loop para encontrar o melhor modelo para cada 'Sub_Prefeitura' e 'Zona'
for sub_prefeitura in Sub_Prefeituras:
    print("Ajustando modelos para Sub-Prefeitura de " + sub_prefeitura)
    best_models_subpref = grid_search_best_model_no_parallel(df_subpref, 'Sub_Prefeitura', sub_prefeitura)
    with open(f'best_models_subpref_{sub_prefeitura}.pkl', 'wb') as f:
        pickle.dump(best_models_subpref, f)

print("Modelos treinados e salvos com sucesso.")        

Ajustando modelos para Sub-Prefeitura de ARICANDUVA-FORMOSA-CARRAO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Sub-Prefeitura de BUTANTA
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Sub-Prefeitura de CAMPO LIMPO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Sub-Prefeitura de CAPELA DO SOCORRO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Sub-Prefeitura de CASA VERDE-CACHOEIRINHA
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Sub-Prefeitura de CIDADE ADEMAR
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Sub-Prefeitura de ERMELINO MATARAZZO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries tempora

In [10]:
for zona in Zonas:
    print("Ajustando modelos para Zona " + zona)
    best_models_zona = grid_search_best_model(df_zonas, 'Zona', zona)
    with open(f'best_models_zona_{zona}.pkl', 'wb') as f:
        pickle.dump(best_models_zona, f)

print("Modelos treinados e salvos com sucesso.")

Ajustando modelos para Zona CENTRO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Zona LESTE
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Zona NORTE
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Zona OESTE
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Ajustando modelos para Zona SUL
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais


/home/cdsw/.local/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/cdsw/.local/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Modelos treinados e salvos com sucesso.
